# Task Graphs Problem Formulation

In [ ]:
using LightGraphs, MetaGraphs, GraphUtils
using Parameters
using LinearAlgebra
using Compose
using Colors
using TaskGraphs
using GraphPlottingBFS
using DataStructures
using JuMP, MathOptInterface
# using GLPK
using Gurobi

# Setup Problem

***
The `project_spec` describes:
1. the inputs, outputs and process time of operations that must be performed at manufacturing stations 
2. the dependencies between those operation (e.g. op 3 requires the outputs of op 1 and op 2

In [ ]:
N = 120                  # num robots
M = 120                  # num delivery tasks
# project_spec = construct_random_project_spec(M;max_parents=3,depth_bias=1.0,Δt_min=0,Δt_max=0)
project_spec1 = construct_random_project_spec(Int(M/2);max_parents=3,depth_bias=0.25,Δt_min=0,Δt_max=0)
project_spec2 = construct_random_project_spec(Int(M/2);max_parents=3,depth_bias=0.25,Δt_min=0,Δt_max=0)
project_spec = combine_project_specs([project_spec1, project_spec2])
plot_graph_bfs(project_spec.graph;mode=nothing,fillcolor="orange")

***
The `delivery_graph` describes the delivery tasks that must be completed by robots in order to meet preconditions (objects 1 and 2 at station 3) for the prescribed operations

In [ ]:
delivery_graph = construct_delivery_graph(project_spec,M)
r₀,s₀,sₜ = initialize_random_2D_task_graph_env(N,M;d=[400,400])
Drs, Dss = cached_pickup_and_delivery_distances(r₀,s₀,sₜ)
G = delivery_graph.graph
# initialize vector of operation times
Δt = zeros(nv(G)) # Δt[j] is the wait time for the object j to appear once all inputs have been satisfied
for op in project_spec.operations
    for id in get_output_ids(op)
        Δt[id] = duration(op)
    end
end
plot_graph_bfs(delivery_graph.graph;mode=nothing)

In [ ]:
# set initial conditions
to0_ = Dict{Int,Float64}()
for v in vertices(G)
    if is_leaf_node(G,v)
        to0_[v] = 0.0
    end
end
tr0_ = Dict{Int,Float64}()
for i in 1:N
    tr0_[i] = 0.0
end

# Solve as MILP

In [ ]:
model = formulate_JuMP_optimization_problem(G,Drs,Dss,Δt,to0_,tr0_,Gurobi.Optimizer;OutputFlag=0);
start_time = time()
optimize!(model)
solve_time = time() - start_time
optimal = (termination_status(model) == MathOptInterface.TerminationStatusCode(1))
@show solve_time
@show optimal;
assignment = Matrix{Int}(value.(model[:x]));

## Process solution (compute slack and convert assignment matrix into task sequences)

In [ ]:
spec = TaskGraphProblemSpec(N,M,G,Drs,Dss,Δt,tr0_,to0_)
cache = SearchCache(N,M,to0_,tr0_)
for j in 1:M
    i = findfirst(assignment[:,j] .== 1)
    cache.x[i,j] = 1
end
cache = process_solution(model,cache,spec);